In [ ]:
from qick import *
import time
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [ ]:
class ContinousWaveProgram(QickProgram):
    def initialize(self):
        self.declare_gen(ch=self.cfg["gen_ch"], nqz=1) # check the nyquist zone of our DAC
        freq_reg = self.freq2reg(self.cfg["freq"], gen_ch=self.cfg["gen_ch"])

        gain_reg = self.cfg["gain"]

        self.set_pulse_registers(ch=self.cfg["gen_ch"],
            freq=freq_reg,
            gain=gain_reg,
            length=10,  # A small length is sufficient for continuous DDS output
            mode=1,     # Repeat the current waveform indefinitely
            outsel=1, # Use DDS directly for a continuous sinusoid
        ) 

        self.synci(200)

    def body(self):
        # Fire the pulse at time 0. Because mode=1 is set, it will continue to stream.
        self.pulse(ch=self.cfg["gen_ch"], t=0)

        # Since it's a continuous wave, we don't have a specific "readout" or "end" to wait for within a loop.
        # If this were part of a larger program, you'd have more complex timing.
        # For continuous output, the program will run and the wave will persist until the board is reset
        # or a new program is loaded/run that overwrites this output.
        
        # We'll include a long sync_all to keep the tProcessor from looping quickly,
        # but the wave itself will continue to output.
        self.sync_all(self.us2cycles(1000)) # Sync for 1000 us (1 ms)




_IncompleteInputError: incomplete input (1265942576.py, line 1)

In [ ]:
GEN_CH = 6

config = {
    "gen_ch": GEN_CH,
    "freq": 2870.0,  # Desired frequency in MHz (2.87 GHz)
    "gain": 5000    # A safe gain value (out of max 32767). Adjust as needed, but start conservative.
}

soc = QickSoc()
soccfg = soc


In [ ]:
print(f"Attempting to generate continuous wave on channel {config['gen_ch']} at {config['freq']} MHz with gain {config['gain']}.")

prog = ContinuousWaveProgram(soccfg, config)
prog.run()

print("Continuous wave is now being generated. You may need to manually stop the output.")
print("To stop, you can run `soc.reset_gens()` or load a new program that sets gain to 0.")

time.sleep(5) # Run for 5 seconds

# To stop the generators manually later in your script or in a new cell:
soc.reset_gens()
print("Generators reset.")
